In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import utils
import importlib

importlib.reload(utils)

<module 'utils' from '/Users/mattgc/code/match-importance/utils.py'>

# Import Data

In [2]:
data = pd.read_csv('data/24_25.csv')
data["Date"] = pd.to_datetime(data["Date"], format="%d/%m/%Y")
data = data.sort_values(by=["Date", "Time"]).reset_index(drop=True)
t = len(data)
data

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,BFECAHH,BFECAHA
0,E0,2024-08-16,20:00,Man United,Fulham,1,0,H,0,0,...,1.86,2.07,1.83,2.11,1.88,2.11,1.82,2.05,1.90,2.08
1,E0,2024-08-17,12:30,Ipswich,Liverpool,0,2,A,0,0,...,2.05,1.88,2.04,1.90,2.20,2.00,1.99,1.88,2.04,1.93
2,E0,2024-08-17,15:00,Arsenal,Wolves,2,0,H,1,0,...,2.02,1.91,2.00,1.90,2.05,1.93,1.99,1.87,2.02,1.96
3,E0,2024-08-17,15:00,Everton,Brighton,0,3,A,0,1,...,1.87,2.06,1.86,2.07,1.92,2.10,1.83,2.04,1.88,2.11
4,E0,2024-08-17,15:00,Newcastle,Southampton,1,0,H,1,0,...,1.87,2.06,1.88,2.06,1.89,2.10,1.82,2.05,1.89,2.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,E0,2024-12-04,19:30,Southampton,Chelsea,1,5,A,1,3,...,2.07,1.86,2.07,1.87,2.07,1.90,2.01,1.86,2.07,1.92
136,E0,2024-12-04,20:15,Arsenal,Man United,2,0,H,0,0,...,2.07,1.86,2.07,1.86,2.07,1.93,1.99,1.87,2.06,1.93
137,E0,2024-12-04,20:15,Aston Villa,Brentford,3,1,H,3,0,...,1.85,2.08,1.85,2.08,1.88,2.08,1.84,2.02,1.92,2.07
138,E0,2024-12-05,19:30,Fulham,Brighton,3,1,H,1,0,...,1.94,1.99,1.94,1.98,1.96,2.00,1.92,1.96,1.97,2.01


In [3]:
remaining = pd.read_csv('data/24_25_remaining.csv')
remaining

,HomeTeam,AwayTeam
0,Everton,Liverpool
1,Aston Villa,Southampton
2,Brentford,Newcastle
3,Crystal Palace,Man City
4,Man Utd,Nott'm Forest
...,...,...
235,Newcastle United,Everton
236,Nottingham Forest,Chelsea
237,Southampton,Arsenal
238,Spurs,Brighton


In [4]:
# Concatenate remaining data to the original data
data = pd.concat([data, remaining], ignore_index=True)
data

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,BFECAHH,BFECAHA
0,E0,2024-08-16,20:00,Man United,Fulham,1.0,0.0,H,0.0,0.0,...,1.86,2.07,1.83,2.11,1.88,2.11,1.82,2.05,1.90,2.08
1,E0,2024-08-17,12:30,Ipswich,Liverpool,0.0,2.0,A,0.0,0.0,...,2.05,1.88,2.04,1.90,2.20,2.00,1.99,1.88,2.04,1.93
2,E0,2024-08-17,15:00,Arsenal,Wolves,2.0,0.0,H,1.0,0.0,...,2.02,1.91,2.00,1.90,2.05,1.93,1.99,1.87,2.02,1.96
3,E0,2024-08-17,15:00,Everton,Brighton,0.0,3.0,A,0.0,1.0,...,1.87,2.06,1.86,2.07,1.92,2.10,1.83,2.04,1.88,2.11
4,E0,2024-08-17,15:00,Newcastle,Southampton,1.0,0.0,H,1.0,0.0,...,1.87,2.06,1.88,2.06,1.89,2.10,1.82,2.05,1.89,2.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,NaN,NaT,NaN,Newcastle United,Everton,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
376,NaN,NaT,NaN,Nottingham Forest,Chelsea,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
377,NaN,NaT,NaN,Southampton,Arsenal,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
378,NaN,NaT,NaN,Spurs,Brighton,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
to_predict = pd.read_csv('data/epl-test.csv')
to_predict

,Date,HomeTeam,AwayTeam
0,01-Feb-25,AFC Bournemouth,Liverpool
1,01-Feb-25,Arsenal,Man City
2,01-Feb-25,Brentford,Spurs
3,01-Feb-25,Chelsea,West Ham
4,01-Feb-25,Everton,Leicester City
5,01-Feb-25,Ipswich Town,Southampton
6,01-Feb-25,Man Utd,Crystal Palace
7,01-Feb-25,Newcastle,Fulham
8,01-Feb-25,Nottingham Forest,Brighton
9,01-Feb-25,Wolves,Aston Villa


In [6]:
t_k = data[(data["HomeTeam"] == "AFC Bournemouth") & (data["AwayTeam"] == "Liverpool")].index[0]
print(t_k)
data.iloc[t_k]

230


Div                     NaN
Date                    NaT
Time                    NaN
HomeTeam    AFC Bournemouth
AwayTeam          Liverpool
                 ...       
MaxCAHA                 NaN
AvgCAHH                 NaN
AvgCAHA                 NaN
BFECAHH                 NaN
BFECAHA                 NaN
Name: 230, Length: 120, dtype: object

# Run a single simulation

In [7]:
standings = utils.calculate_standings(data, 0, t)
standings

,Team,Points
0,Liverpool,35
1,Arsenal,28
2,Chelsea,28
3,Man City,26
4,Brighton,23
5,Fulham,22
6,Nott'm Forest,22
7,Aston Villa,22
8,AFC Bournemouth,21
9,Newcastle,20


In [8]:
simulated_standings = utils.run_simulation(data, standings, t, t_k).sort_values(by="Points", ascending=False).reset_index(drop=True)
simulated_standings

,Team,Points
0,Liverpool,67
1,Chelsea,61
2,Man City,60
3,AFC Bournemouth,59
4,Arsenal,53
5,Everton,50
6,Brentford,48
7,Brighton,48
8,Fulham,46
9,Aston Villa,46


In [9]:
utils.calculate_outcomes(simulated_standings, "AFC Bournemouth", "Liverpool")

(0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1)

# Predict match importance for match $t+k$ with repeated simulations

In [10]:
data = utils.calculate_match_importance(
    data,
    t_k,
    t,
    nruns=100,
)

In [11]:
data.iloc[t_k]

Div                     NaN
Date                    NaT
Time                    NaN
HomeTeam    AFC Bournemouth
AwayTeam          Liverpool
                 ...       
AvgCAHA                 NaN
BFECAHH                 NaN
BFECAHA                 NaN
HI                     0.01
AI                     0.02
Name: 230, Length: 122, dtype: object